In [1]:
import psycopg2
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import dash
import json
import time
from fastapi.responses import JSONResponse
import nest_asyncio
from uvicorn import Config, Server
from fastapi import FastAPI, Query
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from dash import Dash, dcc, html, Input, Output, callback
from flask import Flask, request, jsonify
from flask_cors import CORS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from urllib.parse import quote
import subprocess
import numpy as np
import os
from fastapi import FastAPI, Query, Request
from pydantic import BaseModel
import uvicorn


In [2]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3001"],
)

In [3]:
def run_llama_model(prompt):
    os.chdir("/home/llama/llama.cpp")
    model_path = "models/7B/ggml-model-f16.gguf"
    
    command = [
        "./main",
        "-m", model_path,
        "-p", prompt
    ]
    
    result = subprocess.run(command, capture_output=True, text=True)
    
    return result.stdout

def format_prompt(user_prompt):
    print(user_prompt)
    formatted_prompt = f"<s> [INST] <<SYS>>You are an assistant.<</SYS>> {user_prompt} [/INST]"
    return formatted_prompt

@app.get("/ai")
async def answer(request: Request):
    user_question = request.query_params.get("answer")
    if not user_question:
        return {"error": "Question parameter required"}
    
    formatted_prompt = format_prompt(user_question)
    response = run_llama_model(formatted_prompt)
    parsed_response = response.split("[/INST]")[-1].strip()
    
    return {"response": parsed_response}

if __name__ == "__main__":
    nest_asyncio.apply()
    config = Config(app=app, port=3335)
    server = Server(config)
server.run()

INFO:     Started server process [203837]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 3335): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


AttributeError: 'tuple' object has no attribute 'tb_frame'